# Tema 12: Analizadores sintácticos funcionales

[José A. Alonso](https://www.cs.us.es/~jalonso)  
[Departamento de Ciencias de la Computación e I.A.](https://www.cs.us.es)  
[Universidad de Sevilla](http://www.us.es)  
Sevilla, 8 de agosto de 2019 

> __Notas:__ 
+ La versión interactiva de este tema se encuentra en [Binder](https://mybinder.org/v2/gh/jaalonso/Temas_interactivos_de_PF_con_Haskell/master?urlpath=lab/tree/temas/Tema-12.ipynb).
+ Se desactiva el [corrector estilo de Haskell](https://github.com/gibiansky/IHaskell/wiki#opt-no-lint) en Jupyter.

In [1]:
:opt no-lint

**Librerías auxiliares**

+ En este tema se usan las siguientes librerías:

In [2]:
import Test.QuickCheck
import Data.Char

Analizadores sintácticos
========================

**Analizadores sintácticos**

+ Un *analizador sintáctico* es un programa que analiza textos para determinar
  su *estructura sintáctica*.

+ Ejemplo de análisis sintáctico aritmético: La estructura sintáctica de la
  cadena `"2*3+4"` es el árbol

![](fig/tema-12-fig-1.png)

+ El análisis sintáctico forma parte del preprocesamiento en la mayoría de las
  aplicaciones reales.

El tipo de los analizadores sintácticos
=======================================

**Opciones para el tipo de los analizadores sintácticos**

+ Opción inicial:

```haskell
type Analizador = String -> Tree 
```

+ Con la parte no analizada:

```haskell
type Analizador = String -> (Tree,String)
```

+ Con todos los análisis:

```haskell
type Analizador = String -> [(Tree,String)]
```

+ Con estructuras arbitrarias:

In [3]:
type Analizador a = String -> [(a,String)] 

+ Simplificación: analizadores que fallan o sólo dan un análisis.

Analizadores sintácticos básicos
================================

**Analizadores sintácticos básicos: resultado**

+ `(analiza a cs)` analiza la cadena `cs` mediante el analizador `a`. 

In [4]:
analiza :: Analizador a -> String -> [(a,String)]
analiza a cs = a cs

+ El analizador `resultado v` siempre tiene éxito, devuelve `v` y no consume
  nada. Por ejemplo,

```sesion
ghci> analiza (resultado 1) "abc"
[(1,"abc")]
```

In [5]:
resultado :: a -> Analizador a
resultado v =  \xs -> [(v,xs)]

In [6]:
analiza (resultado 1) "abc"

[(1,"abc")]

**Analizadores sintácticos básicos: fallo**

+ El analizador `fallo` siempre falla. Por ejemplo,

```sesion
ghci> analiza fallo "abc"
[]
```

In [7]:
fallo :: Analizador a
fallo =  \xs -> []

In [8]:
analiza fallo "abc"

[]

**Analizadores sintácticos básicos: elemento**

+ El analizador `elemento` falla si la cadena es vacía y consume el primer
  elemento en caso contrario. Por ejemplo,

```sesion
ghci> analiza elemento ""
[]
ghci> analiza elemento "abc"
[('a',"bc")]
```

In [9]:
elemento :: Analizador Char
elemento =  \xs -> case xs of
                     [] -> []
                     (x:xs) -> [(x , xs)]

In [10]:
analiza elemento ""

[]

In [11]:
analiza elemento "abc"

[('a',"bc")]

Composición de analizadores sintácticos
=======================================

Secuenciación de analizadores sintácticos
-----------------------------------------

+ `((p >*> f) e)` falla si el análisis de `e` por `p` falla, en caso contrario,
  se obtiene un valor (`v`) y una salida (`s`), se aplica la función `f` al
  valor `v` obteniéndose un nuevo analizador con el que se analiza la salida
  `s`.

In [12]:
infixr 5 >*>

(>*>) :: Analizador a -> (a -> Analizador b) -> Analizador b
p >*> f = \ent -> case analiza p ent of
                       []        -> []
                       [(v,sal)] -> analiza (f v) sal

+ `primeroTercero` es un analizador que devuelve los caracteres primero y
  tercero de la cadena. Por ejemplo,

```sesion
primeroTercero "abel"  ==  [(('a','e'),"l")]
primeroTercero "ab"    ==  []
```

In [13]:
primeroTercero :: Analizador (Char,Char)
primeroTercero = 
    elemento >*> \x ->
    elemento >*> \_ ->
    elemento >*> \y ->
    resultado (x,y)

In [14]:
primeroTercero "abel"  

[(('a','e'),"l")]

In [15]:
primeroTercero "ab"    

[]

Elección de analizadores sintácticos
------------------------------------

+ `((p +++ q) e)` analiza `e` con `p` y si falla analiza `e` con `q`. Por
  ejemplo,

```sesion
ghci> analiza (elemento +++ resultado 'd') "abc"  
[('a',"bc")]
ghci> analiza (fallo +++ resultado 'd') "abc"     
[('d',"abc")]
ghci> analiza (fallo +++ fallo) "abc"             
[]
```

In [16]:
(+++) :: Analizador a -> Analizador a -> Analizador a
p +++ q = \ent -> case analiza p ent of
                    []        -> analiza q ent
                    [(v,sal)] -> [(v,sal)]

In [17]:
analiza (elemento +++ resultado 'd') "abc"  

[('a',"bc")]

In [18]:
analiza (fallo +++ resultado 'd') "abc"     

[('d',"abc")]

In [19]:
analiza (fallo +++ fallo) "abc"  

[]

Primitivas derivadas
====================

+ `(sat p)` es el analizador que consume un elemento si dicho elemento cumple
  la propiedad `p` y falla en caso contrario. Por ejemplo,

```sesion
analiza (sat isLower) "hola"  ==  [('h',"ola")]
analiza (sat isLower) "Hola"  ==  []
```

In [20]:
sat :: (Char -> Bool) -> Analizador Char
sat p = elemento >*> \x ->
        if p x then resultado x else fallo

In [21]:
analiza (sat isLower) "hola"  

[('h',"ola")]

In [22]:
analiza (sat isLower) "Hola"  

[]

+ `digito` analiza si el primer carácter es un dígito. Por ejemplo,

```sesion
analiza digito "123"  ==  [('1',"23")]
analiza digito "uno"  ==  []  
```

In [23]:
digito :: Analizador Char
digito = sat isDigit

In [24]:
analiza digito "123"  ==  [('1',"23")]

True

In [25]:
analiza digito "uno"  ==  []

True

+ `minuscula` analiza si el primer carácter es una letra minúscula. Por
  ejemplo,

```sesion
analiza minuscula "eva"  ==  [('e',"va")]
analiza minuscula "Eva"  ==  []
```

In [26]:
minuscula :: Analizador Char
minuscula = sat isLower

In [27]:
analiza minuscula "eva"  ==  [('e',"va")]

True

In [28]:
analiza minuscula "Eva"  ==  []

True

+ `mayuscula` analiza si el primer carácter es una letra mayúscula. Por
  ejemplo,

```sesion
analiza mayuscula "Eva"  ==  [('E',"va")]
analiza mayuscula "eva"  ==  []
```

In [29]:
mayuscula :: Analizador Char
mayuscula = sat isUpper

In [30]:
analiza mayuscula "Eva"  

[('E',"va")]

In [31]:
analiza mayuscula "eva"  

[]

+ `letra` analiza si el primer carácter es una letra. Por ejemplo,

```sesion
analiza letra "Eva"  ==  [('E',"va")]
analiza letra "eva"  ==  [('e',"va")]
analiza letra "123"  ==  []
```

In [32]:
letra :: Analizador Char
letra = sat isAlpha

In [33]:
analiza letra "Eva"  

[('E',"va")]

In [34]:
analiza letra "eva"  

[('e',"va")]

In [35]:
analiza letra "123"  

[]

+ `alfanumerico` analiza si el primer carácter es una letra o un número. Por
  ejemplo,

```sesion
analiza alfanumerico "Eva"   ==  [('E',"va")]
analiza alfanumerico "eva"   ==  [('e',"va")]
analiza alfanumerico "123"   ==  [('1',"23")]
analiza alfanumerico " 123"  ==  []
```

In [36]:
alfanumerico :: Analizador Char
alfanumerico = sat isAlphaNum

In [37]:
analiza alfanumerico "Eva"   

[('E',"va")]

In [38]:
analiza alfanumerico "eva"   

[('e',"va")]

In [39]:
analiza alfanumerico "123"   

[('1',"23")]

In [40]:
analiza alfanumerico " 123"  

[]

+ `(caracter x)` analiza si el primer carácter es igual al carácter `x`. Por
  ejemplo,

```sesion
analiza (caracter 'E') "Eva"  ==  [('E',"va")]
analiza (caracter 'E') "eva"  ==  []
```

In [41]:
caracter :: Char -> Analizador Char
caracter x = sat (== x)

In [42]:
analiza (caracter 'E') "Eva"  

[('E',"va")]

In [43]:
analiza (caracter 'E') "eva"  

[]

+ `(cadena c)` analiza si empieza con la cadena `c`. Por ejemplo,

```sesion
analiza (cadena "abc") "abcdef"  ==  [("abc","def")]
analiza (cadena "abc") "abdcef"  ==  []
```

In [44]:
cadena :: String -> Analizador String
cadena []     = resultado []
cadena (x:xs) = caracter x >*> \x  ->
                cadena xs  >*> \xs ->
                resultado (x:xs)

In [45]:
analiza (cadena "abc") "abcdef"  

[("abc","def")]

In [46]:
analiza (cadena "abc") "abdcef"  

[]

+ `varios p` aplica el analizador `p` cero o más veces. Por ejemplo,

```sesion
analiza (varios digito) "235abc"  ==  [("235","abc")]
analiza (varios digito) "abc235"  ==  [("","abc235")]
```

+ `varios1 p` aplica el analizador `p` una o más veces. Por ejemplo,

```sesion
analiza (varios1 digito) "235abc"  ==  [("235","abc")]
analiza (varios1 digito) "abc235"  ==  []
```

In [47]:
varios :: Analizador a -> Analizador [a]
varios p  = varios1 p +++ resultado []

varios1 :: Analizador a -> Analizador [a]
varios1 p = p        >*> \v  ->
            varios p >*> \vs ->
            resultado (v:vs)

In [48]:
analiza (varios digito) "235abc"  

[("235","abc")]

In [49]:
analiza (varios digito) "abc235"  

[("","abc235")]

In [50]:
analiza (varios1 digito) "235abc"  

[("235","abc")]

In [51]:
analiza (varios1 digito) "abc235"  

[]

+ `ident` analiza si comienza con un identificador (i.e. una cadena que
  comienza con una letra minúscula seguida por caracteres alfanuméricos). Por
  ejemplo,

```sesion
ghci> analiza ident "lunes12 de Ene"  
[("lunes12"," de Ene")]
ghci> analiza ident "Lunes12 de Ene"  
[]
```

In [52]:
ident :: Analizador String
ident =  minuscula           >*> \x  ->
         varios alfanumerico >*> \xs ->
         resultado (x:xs)

In [53]:
analiza ident "lunes12 de Ene"  

[("lunes12"," de Ene")]

In [54]:
analiza ident "Lunes12 de Ene"  

[]

+ `nat` analiza si comienza con un número natural. Por ejemplo,

```sesion
analiza nat "14DeAbril"   ==  [(14,"DeAbril")]
analiza nat " 14DeAbril"  ==  []
```

In [55]:
nat :: Analizador Int
nat = varios1 digito >*> \xs ->
      resultado (read xs)

In [56]:
analiza nat "14DeAbril"   

[(14,"DeAbril")]

In [57]:
analiza nat " 14DeAbril"  

[]

+ `espacio` analiza si comienza con espacios en blanco. Por ejemplo,

```sesion
analiza espacio "    a b c"  ==  [((),"a b c")]
```

In [58]:
espacio :: Analizador ()
espacio = varios (sat isSpace) >*> \_ -> 
          resultado ()

In [59]:
analiza espacio "    a b c"

[((),"a b c")]

Tratamiento de los espacios
===========================

+ `unidad p` ignora los espacios en blanco y aplica el analizador `p`. Por
  ejemplo,

```sesion
ghci> analiza (unidad nat) " 14DeAbril"     
[(14,"DeAbril")]
ghci> analiza (unidad nat) " 14   DeAbril"  
[(14,"DeAbril")]
```

In [60]:
unidad :: Analizador a -> Analizador a
unidad p = espacio >*> \_ ->
           p       >*> \v ->
           espacio >*> \_ ->
           resultado v

In [61]:
analiza (unidad nat) " 14DeAbril"     

[(14,"DeAbril")]

In [62]:
analiza (unidad nat) " 14   DeAbril"  

[(14,"DeAbril")]

+ `identificador` analiza un identificador ignorando los espacios delante y
  detrás. Por ejemplo,

```sesion
ghci> analiza identificador "  lunes12  de Ene"
[("lunes12","de Ene")]  
```

In [63]:
identificador :: Analizador String
identificador = unidad ident

In [64]:
analiza identificador "  lunes12  de Ene"

[("lunes12","de Ene")]

+ `natural` analiza un número natural ignorando los espacios delante y
  detrás. Por ejemplo,

```sesion
analiza natural "  14DeAbril"  ==  [(14,"DeAbril")]  
```

In [65]:
natural :: Analizador Int
natural =  unidad nat

In [66]:
analiza natural "  14DeAbril"

[(14,"DeAbril")]

+ `(simbolo xs)` analiza la cadena `xs` ignorando los espacios delante y
  detrás. Por ejemplo,

```sesion
ghci> analiza (simbolo "abc") "  abcdef"  
[("abc","def")]  
```

In [67]:
simbolo :: String -> Analizador String
simbolo xs =  unidad (cadena xs)

In [68]:
analiza (simbolo "abc") "  abcdef" 

[("abc","def")]

+ `listaNat` analiza una lista de naturales ignorando los espacios. Por
  ejemplo,

```sesion
ghci> analiza listaNat " [  2,  3, 5   ]"  
[([2,3,5],"")]
ghci> analiza listaNat " [  2,  3,]"       
[]
```

In [69]:
listaNat :: Analizador [Int]
listaNat = simbolo "["          >*> \_ ->
           natural              >*> \n ->
           varios (simbolo ","  >*> \_ ->
                   natural)     >*> \ns ->
           simbolo "]"          >*> \_ ->
           resultado (n:ns)

In [70]:
analiza listaNat " [  2,  3, 5   ]"  

[([2,3,5],"")]

In [71]:
analiza listaNat " [  2,  3,]"

[]

Analizador de expresiones aritméticas
=====================================

**Expresiones aritméticas**

+ Consideramos expresiones aritméticas:
    + construidas con números, operaciones (`+` y `*`) y paréntesis.
    + `+` y `*` asocian por la derecha.
    + `*` tiene más prioridad que `+`. 

+ Ejemplos:
    + `2+3+5` representa a `2+(3+5)`.
    + `2*3+5` representa a `(2*3)+5`.


**Gramáticas de las expresiones aritméticas: Gramática 1**

+ Gramática 1 de las expresiones aritméticas:

```sesion
expr ::=  expr + expr | expr * expr | (expr) | nat 
nat  ::=  0 | 1 | 2 | ...
```

+ La gramática 1 no considera prioridad: acepta `2+3*5` como `(2+3)*5` y como
  `2+(3*5)`

+ La gramática 1 no considera asociatividad: acepta `2+3+5` como `(2+3)+5` y
  como `2+(3+5)`

+ La gramática 1 es ambigua.


**Gramáticas de las expresiones aritméticas: Gramática 2**

+ Gramática 2 de las expresiones aritméticas (con prioridad):

```sesion
expr    ::=  expr + expr | term 
term    ::=  term * term | factor 
factor  ::=  (expr) | nat 
nat     ::=  0 | 1 | 2 | dots
```

+ La gramática 2 sí considera prioridad: acepta `2+3*5` sólo como `2+(3*5)`

+ La gramática 2 no considera asociatividad: acepta `2+3+5` como `(2+3)+5` y
como `2+(3+5)`

+ La gramática 2 es ambigua.


**Árbol de análisis sintáctico de `2*3+5` con la gramática 2**

![](fig/tema-12-fig-2.png)

**Gramáticas de las expresiones aritméticas: Gramática 3**

+ Gramática 3 de las expresiones aritméticas:

```sesion
expr   ::= term + expr | term 
term   ::= factor * term | factor 
factor ::= (expr) | nat 
nat    ::= 0 | 1 | 2 | dots
```

+ La gramática 3 sí considera prioridad: acepta `2+3*5` sólo como `2+(3*5)`

+ La gramática 3 sí considera asociatividad: acepta `2+3+5` como `2+(3+5)`

+ La gramática 3 no es ambigua (i.e. es libre de contexto).


**Árbol de análisis sintáctico de `2+3+5` con la gramática 3**

![](fig/tema-12-fig-3.png)

**Gramáticas de las expresiones aritméticas: Gramática 4**

+ La gramática 4 se obtiene simplificando la gramática 3:

```sesion
expr   ::= term (+ expr | epsilon) 
term   ::= factor (* term | epsilon) 
factor ::= (expr) | nat 
nat    ::= 0 | 1 | 2 | dots
```

  <p class="indent">donde ε es la cadena vacía.</p>

+ La gramática 4 no es ambigua.

+ La gramática 4 es la que se usará para escribir el analizador de expresiones
  aritméticas.


**Analizador de expresiones aritméticas**

In [72]:
-- expr analiza una expresión aritmética devolviendo su valor. Por ejemplo,
--    analiza expr "2*3+5"     ==  [(11,"")]
--    analiza expr "2*(3+5)"   ==  [(16,"")]
--    analiza expr "2+3*5"     ==  [(17,"")]
--   analiza expr "2*3+5abc"  ==  [(11,"abc")]
expr :: Analizador Int
expr = term              >*> \t ->
       (simbolo "+"      >*> \_ ->              
        expr             >*> \e ->
        resultado (t+e))
       +++ resultado t

-- term analiza un término de una expresión aritmética devolviendo su
-- valor. Por ejemplo,
--    analiza term "2*3+5"      ==  [(6,"+5")]
--    analiza term "2+3*5"      ==  [(2,"+3*5")]
--    analiza term "(2+3)*5+7"  ==  [(25,"+7")]
term :: Analizador Int
term =  factor            >*> \f ->
        (simbolo "*"      >*> \_ ->
         term             >*> \t ->
         resultado (f*t))
        +++ resultado f

-- factor analiza un factor de una expresión aritmética devolviendo su
-- valor. Por ejemplo,
--    analiza factor "2*3+5"      ==  [(2,"*3+5")]
--    analiza factor "(2+3)*5"    ==  [(5,"*5")]
--    analiza factor "(2+3*7)*5"  ==  [(23,"*5")]
factor :: Analizador Int
factor =  (simbolo "("  >*> \_ ->
           expr         >*> \e ->
           simbolo ")"  >*> \_ ->
           resultado e)
          +++ natural

+ Ejemplos 

In [73]:
analiza expr "2*3+5"     

[(11,"")]

In [74]:
analiza expr "2*(3+5)"   

[(16,"")]

In [75]:
analiza expr "2+3*5"     

[(17,"")]

In [76]:
analiza expr "2*3+5abc"  

[(11,"abc")]

In [77]:
analiza term "2*3+5"      

[(6,"+5")]

In [78]:
analiza term "2+3*5"      

[(2,"+3*5")]

In [79]:
analiza term "(2+3)*5+7"  

[(25,"+7")]

In [80]:
analiza factor "2*3+5"     

[(2,"*3+5")]

In [81]:
analiza factor "(2+3)*5"   

[(5,"*5")]

In [82]:
analiza factor "(2+3*7)*5" 

[(23,"*5")]

+ `(valor cs)` analiza la cadena `cs` devolviendo su valor si es una expresión
  aritmética y un mensaje de error en caso contrario. Por ejemplo,

```sesion
valor "2*3+5"      ==  11
valor "2*(3+5)"    ==  16
valor "2 * 3 + 5"  ==  11
valor "2*3x"       ==  *** Exception: sin usar x
valor "-1"         ==  *** Exception: entrada no valida
```

In [83]:
valor :: String -> Int
valor xs = case (analiza expr xs) of
             [(n,[])]  -> n
             [(_,sal)] -> error ("sin usar " ++ sal)
             []        -> error "entrada no valida"

In [84]:
valor "2*3+5"      

11

In [85]:
valor "2*(3+5)"    

16

In [86]:
valor "2 * 3 + 5"  

11

In [87]:
valor "2*3x"       

: 

In [88]:
valor "-1"         

: 

Bibliografía
============

+ R. Bird. *Introducción a la programación funcional con Haskell*. Prentice Hall, 2000.
    + Cap. 11: Análisis sintáctico.

+ G. Hutton. *Programming in Haskell*. Cambridge University Press, 2007.
    + Cap. 8: Functional parsers.

+ G. Hutton y E. Meijer.
  [Monadic parser combinators](http://www.cs.nott.ac.uk/~gmh/monparsing.pdf).
  Technical Report NOTTCS-TR-96-4, Department of Computer Science,
  University of Nottingham, 1996.

+ G. Hutton y E. Meijer. 
  [Monadic parsing in Haskell](http://www.cs.nott.ac.uk/~gmh/pearl.pdf).
  Journal of Functional Programming, 8(4): 437-444, 1998.

+ B.C. Ruiz, F. Gutiérrez, P. Guerrero y J.E. Gallardo.
  *Razonando con Haskell*.
  Thompson, 2004.
    + Cap. 14: Analizadores.